In [6]:
# Convert project files to a single prompt to upload to Claud Projects
from pathlib import Path
import fnmatch

## Character Animation Editor

In [2]:
skip_files_folders = {'assets', 'vite-env.d.ts', '.DS_Store'}
src_folder = Path('../src')
include_files = ['../index.html', '../package.json']
design_doc = Path('./Design Document.md')

In [3]:
with open('prompt.md', 'w') as prompt:

    prompt.writelines(["This is the Design Document:", '\n\n'])
    
    # Write design doc
    prompt.write(design_doc.read_text())

    # Write code base
    prompt.writelines(['''
This is the existing code base, each file is defined in the following format:

[Path to the file enclosed in square brackets]
```
Code enclosed in tripple backticks
```

Here are the files:
    ''', '\n\n'])
    
    for f in src_folder.rglob('*'):
        if not any(p for p in f.parts if p in skip_files_folders) and f.is_file():
            prompt.writelines([
                f'[{"/".join(f.parts)}]', '\n\n',
                '```', '\n',
                f.read_text(),
                '```',
                '\n\n',
            ])
    # Additional files
    for f in [Path(f) for f in include_files]:
        prompt.writelines([
            f'[{"/".join(f.parts)}]', '\n\n',
            '```', '\n',
            f.read_text(),
            '```',
            '\n\n',
        ])

In [ ]:
## Crossover

In [19]:
skip_files_folders = {'assets', 'vite-env.d.ts', '.DS_Store', '.env', '.gitignore'}
include_only_files = {'*.ts', '*.js', '*.svelte', '*.md', '*.json'}
dest = Path('./crossover/prompt.md')
folders = [
    # Design Docs
    {
        "header": '''

This are the design document, each file is defined in the following format:

[Path to the file enclosed in square brackets]
```
Document enclosed in tripple backticks
```

Here are the files:

        ''',
        "path": [Path('./crossover')]
    },
    # Code
    {
        "header": '''
    
This is the existing code base, each file is defined in the following format:

[Path to the file enclosed in square brackets]
```
Code enclosed in tripple backticks
```

Here are the files:

        ''',
        "path": [
            Path('../christmas/app/src/lib/crossover'),
            Path('../christmas/app/src/lib/components/crossover'),
            Path('../christmas/app/src/lib/server/crossover'),
            
        ]
    },
    
]

In [20]:

def should_include_file(file_path):
    return (
        file_path.is_file() and
        not any(p for p in file_path.parts if p in skip_files_folders) and
        any(fnmatch.fnmatch(file_path.name, pattern) for pattern in include_only_files)
    )

with open(dest, 'w') as prompt:
    for folder in folders:
        prompt.write(folder["header"])
        prompt.write('\n')
        
        for path in folder["path"]:
            for file_path in path.rglob('*'):
                if should_include_file(file_path):
                    relative_path = file_path.relative_to(path)
                    prompt.writelines([
                        f'[{relative_path}]\n\n',
                        '```\n',
                        file_path.read_text(),
                        '\n```\n\n'
                    ])